## Setup

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pyarrow.feather as feather

# W & B test

In [29]:
import wandb
from wandb.keras import WandbCallback
!wandb login


wandb: Currently logged in as: uwe-sterr (use `wandb login --relogin` to force relogin)


In [2]:
fluxData_df = feather.read_feather('data/fluxData.feather')
zernikeData_df = feather.read_feather('data/zernikeData.feather')
fluxData_df


,FluxCoeef1,FluxCoeef2,FluxCoeef3,FluxCoeef4,FluxCoeef5,FluxCoeef6,FluxCoeef7,FluxCoeef8,FluxCoeef9,FluxCoeef10,FluxCoeef11,FluxCoeef12,FluxCoeef13,FluxCoeef14,FluxCoeef15,FluxCoeef16,FluxCoeef17,FluxCoeef18,FluxCoeef19
0,5.813705,1.143091,0.531734,0.730568,0.927093,1.325805,0.477268,0.649769,1.529369,0.363719,1.094243,0.320111,0.827265,1.110052,0.468520,0.261182,0.385878,0.600388,0.119944
1,4.871158,1.608725,1.189874,0.765206,1.017840,0.860749,1.256996,1.183990,0.197036,0.936330,1.008980,0.800554,0.442897,0.560865,0.376131,0.432417,0.253278,0.431240,0.303724
2,5.066373,0.574476,0.896186,0.614643,1.189386,0.736408,1.761553,1.132988,0.241487,0.441099,2.297195,1.121753,0.447027,0.178829,0.932022,0.705389,0.186733,0.380705,0.361099
3,6.026372,1.059205,1.390196,1.145955,0.928381,0.664668,0.838922,1.074237,0.696530,1.163540,0.822802,0.670485,0.511729,0.490346,0.925361,0.657651,0.370025,0.372889,0.195592
4,2.000598,1.676623,0.320688,0.283186,0.454243,1.828364,2.426376,0.779883,0.243130,0.742514,1.465755,1.119488,0.682209,0.198723,1.550706,0.457396,0.341759,0.522653,0.525872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58735,6.033478,0.377019,1.349274,0.933555,0.581626,0.655475,1.720409,0.611912,0.608115,0.438212,0.881331,0.692223,0.624412,0.217507,0.906866,0.724129,0.306011,0.930735,0.362653
58736,3.949661,0.900760,0.339916,0.467232,1.239033,1.219449,3.338060,0.993904,0.196614,0.603852,1.311528,0.665667,0.390763,0.456508,0.399422,1.034937,0.402864,0.288359,0.479911
58737,5.692162,0.590152,0.595392,1.047038,1.293543,1.101659,2.561419,0.970390,0.465767,1.043618,0.674526,0.613000,0.360255,0.174299,0.492189,1.086915,0.177230,0.228387,0.266489
58738,3.708839,2.601630,1.000832,0.880887,0.921476,1.263213,1.445328,1.031273,0.382903,1.556901,0.494232,0.564818,0.425112,0.628254,0.456663,0.437302,0.575897,0.563907,0.284540


## Prepare the data

In [8]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


# Build model

In [9]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)               

## Train the model

In [ ]:
# Initialize wandb with your project name
run = wandb.init(project='SimpleMNISTconvnetII',
                 config={  # and include hyperparameters and metadata
                     "learning_rate": 0.05,
                     "epochs": 5,
                     "batch_size": 128,
                     "loss_function": "sparse_categorical_crossentropy",
                     "architecture": "CNN test",
                     "dataset": "CIFAR-10 sydney"
                 })
config = wandb.config  # We'll use this to configure our experiment


#batch_size = 128
#epochs = 15
#config.learning_rate = 0.01
#config.optimzer = "adam"
#... Define a model
# 3. Log layer dimensions and metrics over time
# model.fit(X_train, y_train, validation_data=(X_test, y_test),

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=config.batch_size, epochs=config.epochs, validation_split=0.1, callbacks=[WandbCallback()])



## Sweep
example in colab
https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Organizing_Hyperparameter_Sweeps_in_PyTorch_with_W%26B.ipynb#scrollTo=jnhWMrkodjdN 


In [31]:
# Initialize wandb with your project name
def train():
    with wandb.init() as run:
        config = wandb.config
        model = make_model(config)
        for epoch in range(config["epochs"]):
            loss = model.fit(x_train, y_train, batch_size=config.batch_size, epochs=config.epochs, validation_split=0.1, callbacks=[WandbCallback()])



count = 5 # number of runs to execute
wandb.agent("sh07tp28", function=train, count=count)


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: ERROR Error while calling W&B API: entity not found during createAgent (<Response [404]>)


CommError: entity not found during createAgent

## Evalutate Model

In [26]:
wandb.agent("6khwe1rw")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


CommError: Sweep wandb agent uwe-sterr/SimpleMNISTconvnetII/6khwe1rw not found

In [6]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.024764826521277428
Test accuracy: 0.9922000169754028
